In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import sympy
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),'../..')))

from Rmse_loss import rmse_loss 
from All import all0
from Select import select
from Train_fun import train_fun


torch.set_default_dtype(torch.float32)
device = torch.device('cuda:0')
np.set_printoptions(suppress=True)

In [2]:
class FiveNet_sin(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.linear1 = nn.Linear(ni, 128)
        self.linear2 = nn.Linear(128,128)
        self.linear3 = nn.Linear(128, 64)
        self.linear4 = nn.Linear(64,64)
        self.linear5 = nn.Linear(64,1)

    def forward(self, x):
        x = torch.sin(self.linear1(x))
        x = torch.sin(self.linear2(x))
        x = torch.sin(self.linear3(x))
        x = torch.sin(self.linear4(x))
        x = self.linear5(x)
        return x

In [3]:
def diff_burgers(net,cor_train):
    cache = np.zeros((1,6))
    pts = cor_train.clone().detach().to(device)
    pts.requires_grad_(True)
    n_iter = len(pts)//10000
    for i in range(n_iter+1):
        pts0 = pts[10000*i:10000*(i+1)]
        outs = net(pts0)
        grad = torch.autograd.grad(outs, pts0, grad_outputs=torch.ones_like(outs), create_graph=True)[0]
        ut_t = (grad[:,0]).reshape(-1,1)
        ux_t = (grad[:,1]).reshape(-1,1)
        uxx_t  = ((torch.autograd.grad(ux_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)
        uxxx_t  = ((torch.autograd.grad(uxx_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)
        uxxxx_t  = ((torch.autograd.grad(uxxx_t, pts0, grad_outputs=torch.ones_like(outs), 
                create_graph=True)[0])[:,1]).reshape(-1,1)

        u_pred = outs.cpu().detach().numpy()
        ut = ut_t.cpu().detach().numpy()
        ux = ux_t.cpu().detach().numpy()
        uxx = uxx_t.cpu().detach().numpy()
        uxxx = uxxx_t.cpu().detach().numpy()
        uxxxx = uxxxx_t.cpu().detach().numpy()
        del ut_t, ux_t, uxx_t, uxxx_t, uxxxx_t, outs
        torch.cuda.empty_cache()
        cache = np.vstack((cache,np.hstack((u_pred,ux,uxx,uxxx,uxxxx,ut))))
    return cache[1:]

In [4]:
m = 201; n = 501
t = np.linspace(2,8,m); x = np.linspace(-8,8,n)
T,X = np.meshgrid(t,x) 
zz = -4*X/(X**2+2*T)
t = T.reshape(-1,1); x = X.reshape(-1,1)
u = -0.8*x/(x**2+0.4*t)
data = np.hstack((t,x,u)).astype('float32')
data = data[data[:,0]//0.03%3==0]; data = data[data[:,1]//0.012%7==0]

In [5]:
per = 0.2
np.random.seed(0)
u_clean = data[:,-1].reshape(-1,1)
u_noise = u_clean + per*np.std(u_clean)*np.random.randn(u_clean.shape[0],u_clean.shape[1])
u_noise = u_noise.astype('float32')
cor_train = torch.from_numpy(data[:,:-1])
u_train = torch.from_numpy(u_noise).reshape(-1,1)

In [6]:
net = FiveNet_sin(2).to(device)
try:
    net.load_state_dict(torch.load('net/net_{}'.format(per)))
except:
    train_fun(net,cor_train,u_train,N_red_lr=4,epochs=5000,lr=0.001)
    torch.save(net.state_dict(),'net/net_{}'.format(per))

In [7]:
cache = diff_burgers(net,cor_train)
u_pred = cache[:,0].reshape(-1,1)
ux = cache[:,1].reshape(-1,1)
uxx = cache[:,2].reshape(-1,1)
uxxx = cache[:,3].reshape(-1,1)
uxxxx = cache[:,4].reshape(-1,1)
ut = cache[:,-1].reshape(-1,1)

In [8]:
rmse_loss(-u_pred*ux+0.2*uxx,ut)

tensor(0.4957, device='cuda:0', dtype=torch.float64)

In [9]:
feature = np.hstack((u_pred,ux,uxx,uxxx,uxxxx,ut))
LC,exp_list,exp = all0('result_{}.txt'.format(per),feature,state=1,
                              data_points=500,niterations=100,tol=0.05)

/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/julia_helpers.py:162: UserWarning: `torch` was loaded before the Julia instance started. This may cause a segfault when running `PySRRegressor.fit`. To avoid this, please run `pysr.julia_helpers.init_julia()` *before* importing `torch`. For updates, see https://github.com/pytorch/pytorch/issues/78829
  warnings.warn(


Compiling Julia backend...


/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/julia_helpers.py:208: UserWarning: Your system's Python library is static (e.g., conda), so precompilation will be turned off. For a dynamic library, try using `pyenv` and installing with `--enable-shared`: https://github.com/pyenv/pyenv/blob/master/plugins/python-build/README.md#building-with---enable-shared.
  warnings.warn(


Started!
tensor(0.4646, device='cuda:0', dtype=torch.float64)
0 tensor(0.4918, device='cuda:0', dtype=torch.float64)
1 tensor(0.4918, device='cuda:0', dtype=torch.float64)
2 tensor(0.4918, device='cuda:0', dtype=torch.float64)
3 tensor(0.4918, device='cuda:0', dtype=torch.float64)
4 tensor(0.5767, device='cuda:0', dtype=torch.float64)
[ 0.          0.          0.          0.          0.20078758  0.
  0.         -1.00114524  0.          0.          0.09282723  0.
  0.          0.          0.        ]
[[0.         1.         0.02547313 0.02598108 0.00007076 0.00872888]
 [0.         2.         0.04066366 0.03202525 0.00687661 0.00467059]
 [0.         3.         0.00912628 0.00912628 0.00403609 0.00403609]
 [0.         4.         0.02077015 0.02077015 0.00403609 0.00403609]
 [1.         2.         0.04325918 0.01810477 0.00691727 0.00472914]
 [1.         3.         0.04379302 0.04379302 0.00404296 0.00404296]
 [1.         4.         0.10712761 0.10712761 0.00404296 0.00404296]
 [2.        

/home/chr/SI-SR/Symmetry.py:196: RuntimeWarning: invalid value encountered in divide
  results[:, 2:] = results[:, 2:] / (np.sort(results[:, 2:])[:, 1].reshape(-1, 1))
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!
[-0.15601011 -0.38251713  0.07048345  0.07349271  0.07406693]
tensor(0.4866, device='cuda:0', dtype=torch.float64)
0 tensor(0.4928, device='cuda:0', dtype=torch.float64)
1 tensor(0.6890, device='cuda:0', dtype=torch.float64)
2 tensor(0.9472, device='cuda:0', dtype=torch.float64)
3 tensor(0.4928, device='cuda:0', dtype=torch.float64)
[ 0.         -1.02807955  0.21111943  0.          0.        ]
[[0.         1.         0.11423203 0.07530914 0.0071697  0.00487184]
 [0.         2.         0.2238679  0.2238679  0.00415173 0.00415173]
 [0.         3.         0.62179736 0.62179736 0.00415173 0.00415173]
 [1.         2.         0.06118995 0.06118995 0.00349395 0.00349395]
 [1.         3.         0.14290604 0.14290604 0.00349395 0.00349395]
 [2.         3.         0.         0.         0.         0.        ]]
[[0.         1.         0.00487184 0.67950368 3.        ]
 [0.         2.         0.00415173 1.         2.        ]
 [0.         3.         0.00415173 1.         2.        ]
 [1. 

/home/chr/SI-SR/Symmetry.py:196: RuntimeWarning: invalid value encountered in divide
  results[:, 2:] = results[:, 2:] / (np.sort(results[:, 2:])[:, 1].reshape(-1, 1))
/home/chr/anaconda3/envs/pysren/lib/python3.11/site-packages/pysr/sr.py:1346: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Started!
[0.97483317 2.26295912 2.22413385 1.79063243 1.68367796 1.05939307]


In [10]:
k = select(LC,0.1)
exp_list[k]

[ 1  1  1  1 -1 -1 -1 -1 -1 -1 -1]
[ 1  1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1]


-x0 + 0.2107476*x1